## Navigate to the Correct Directory

The following code navigates to the dataprocessing directory.

In [ ]:
cd ../dataprocessing

## Call the import statements

The following code imports the necessary code to run the code in the rest of this notebook.

In [ ]:
# import statements
import audio_processing as ap

import functools
import os
import sys
from absl import logging

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow import feature_column
from tensorflow.keras import layers
from matplotlib import pyplot as plt
import sklearn

# The following lines adjust the granularity of reporting.
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format
# tf.keras.backend.set_floatx('float32')

print("Ran the import statements.")

In [ ]:
# set logging to print logging.INFO logs
logging.set_verbosity(logging.INFO)

In [ ]:
# arguments for audio_processing
src_dir = 'example_src_dir'
dest_dir = 'example_dest_dir'
filename = 'test_set2'
labels = ['Gunshot, gunfire']
available_features = ['chroma_stft',
                       'chroma_cqt',
                       'chroma_cens',
                       'melspectrogram',
                       'mfcc',
                       'rms',
                       'spectral_centroid',
                       'spectral_bandwidth',
                       'spectral_contrast',
                       'spectral_flatness',
                       'spectral_rolloff',
                       'poly_features',
                       'tonnetz',
                       'zero_crossing_rate'
]
features_to_extract = ['rms']

In [ ]:
# Import the dataset
df = ap.output_df(src_dir, dest_dir, filename, labels, features_to_extract, False)

In [ ]:
df.head()

In [ ]:
train_df_rms = df.copy()
train_df_rms.head()

In [ ]:
train_df_rms = train_df_rms.reindex(
    np.random.permutation(train_df_rms.index))
train_df_rms.head()

In [ ]:
# temp bug fix for rows with None features
# also deletes the outer array in the features
rows_with_none = []
print(train_df_rms.size / 2)
# print(train_df_rms.rms[0])
for i in train_df_rms.index:
    if train_df_rms.rms[i] is None:
        print('hey dum dum look at me', i)
        rows_with_none.append(i)
        continue
    train_df_rms.rms[i] = train_df_rms.rms[i][0]
print(rows_with_none)
train_df_rms = train_df_rms.drop(rows_with_none)
train_df_rms.size / 2
train_df_rms.head(10000)

In [ ]:
X = np.array(train_df_rms.rms.tolist(), dtype=object)
y = np.array(train_df_rms.label.tolist())

In [ ]:
# Temp bug fix for having different sized features
temp_x = []
temp_y = []
count = 0
for arr, label in zip(X, y):
    if arr is None or len(arr) < 431:
#         print('hey look at me', len(arr))
        count += 1
        continue
    temp_x.append(arr)
    temp_y.append(label)
X = np.array(temp_x, dtype=object)
y = np.array(temp_y)
print(count)

In [ ]:
x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state = 42)

In [ ]:
print(y_val)
print(y_train)

In [ ]:
# trying to fix bug:
# ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).
# It worked!!!
from keras import backend as K
x_train = K.cast_to_floatx(x_train)
y_train = K.cast_to_floatx(y_train)
x_val = K.cast_to_floatx(x_val)
y_val = K.cast_to_floatx(y_val)

In [ ]:
# Define the plotting function.
def plot_curve(epochs, hist, list_of_metrics):
    """Plot a curve of one or more classification metrics vs. epoch."""  
    # list_of_metrics should be one of the names shown in:
    # https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#define_the_model_and_metrics  

    plt.figure()
    plt.xlabel("Epoch")
    plt.ylabel("Value")

    for m in list_of_metrics:
        x = hist[m]
        plt.plot(epochs[1:], x[1:], label=m)

    plt.legend()
    return plt
    

print("Defined the plot_curve function.")

## Train 0
Deep Neural Network:
* Input Layer: 431 nodes
* Hidden Layer 1: 431 nodes
* Ouput Layer: 1 node

Hyper-parameters:
* Loss Function: BinaryCrossEntropy
* Activation Function: Relu
* Optimizer Function: RMSprop
* Learning Rate: 0.001 
* Epochs: 500
* Batch_Size: 25
* Classification Threshold: 0.7
* Regularization: L2
* Regularization Lambda: 0.01

In [ ]:
# Define the functions that create and train a model.
def create_model(my_learning_rate, my_metrics):
    """Create and compile a simple classification model."""
    # Discard any pre-existing version of the model.
    model = None

    # Most simple tf.keras models are sequential.
    model = tf.keras.models.Sequential()

    # Add the input layer of 431 nodes
    model.add(tf.keras.layers.Dense(units=431, input_shape=(431,)))
              
    # Implement L2 regularization in the first hidden layer.
    model.add(tf.keras.layers.Dense(units=431, 
                                  activation='relu',
                                  kernel_regularizer=tf.keras.regularizers.l2(0.01),
                                  name='Hidden1'))

    # Funnel the regression value through a sigmoid function.
    model.add(tf.keras.layers.Dense(units=1, input_shape=(1,),
                                  activation=tf.sigmoid,
                                  name='Output'))

    # Call the compile method to construct the layers into a model that
    # TensorFlow can execute.  Notice that we're using a different loss
    # function for classification than for regression.    
    model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=my_learning_rate),                                                   
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=my_metrics)

    return model        
              
def train_model(model, features, label, epochs, label_name,
                batch_size=None, my_validation_split=0.0,
                validation_data=None, shuffle=True):
    """Feed a dataset into the model in order to train it."""

    # The x parameter of tf.keras.Model.fit can be a list of arrays.
    history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, shuffle=shuffle, validation_data=validation_data)

    # The list of epochs is stored separately from the rest of history.
    epochs = history.epoch

    # Isolate the classification metric for each epoch.
    hist = pd.DataFrame(history.history)

    return epochs, hist  

print("Defined the create_model and train_model functions.")  

In [254]:
# The following variables are the hyperparameters.
learning_rate = 0.001
epochs = 500
batch_size = 25
classification_threshold = 0.70
label_name = "label"

# Here is the updated definition of METRICS:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(
          name='accuracy', threshold=classification_threshold),
      tf.keras.metrics.Precision(
          thresholds=classification_threshold, name='precision'),
      tf.keras.metrics.Recall(
          thresholds=classification_threshold, name="recall"),
]

# Establish the model's topography.
my_model = create_model(learning_rate, METRICS)

my_model.summary()

# Train the model on the training set.
epochs, hist = train_model(my_model, x_train, y_train, epochs, 
                          label_name, batch_size, validation_data=(x_val, y_val))

# Plot metrics vs. epochs
list_of_metrics_to_plot = ['accuracy', "precision", "recall", 'val_accuracy', 'val_precision', 'val_recall'] 
plot_curve(epochs, hist, list_of_metrics_to_plot)
plot_curve(epochs, hist, ['loss', 'val_loss'])

training_performance =  my_model.evaluate(x_train, y_train, verbose=0)
print('Training Performance')
print('---------------------------------')
print('loss: ', training_performance[0])
print('accuracy: ', training_performance[1])
print('precision: ', training_performance[2])
print('recall: ', training_performance[3])
print()

validation_performance =  my_model.evaluate(x_val, y_val, verbose=0)
print('Validation Performance')
print('---------------------------------')
print('loss: ', validation_performance[0])
print('accuracy: ', validation_performance[1])
print('precision: ', validation_performance[2])
print('recall: ', validation_performance[3])

Model: "sequential_68"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_68 (Dense)             (None, 431)               186192    
_________________________________________________________________
Hidden1 (Dense)              (None, 431)               186192    
_________________________________________________________________
Output (Dense)               (None, 1)                 432       
Total params: 372,816
Trainable params: 372,816
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
350/350 [==============================] - 1s 4ms/step - loss: 0.9684 - accuracy: 0.5400 - precision: 0.6333 - recall: 0.0234 - val_loss: 0.6834 - val_accuracy: 0.5517 - val_precision: 0.7105 - val_recall: 0.0271
Epoch 2/500
350/350 [==============================] - 1s 3ms/step - loss: 0.6900 - accuracy: 0.5488 - precision: 0.6970 - recall: 0.0510 - val_loss: 0.

Epoch 34/500
350/350 [==============================] - 2s 4ms/step - loss: 0.6357 - accuracy: 0.6047 - precision: 0.7819 - recall: 0.2068 - val_loss: 0.7031 - val_accuracy: 0.5838 - val_precision: 0.6257 - val_recall: 0.2151
Epoch 35/500
350/350 [==============================] - 2s 5ms/step - loss: 0.6344 - accuracy: 0.6089 - precision: 0.7841 - recall: 0.2183 - val_loss: 0.6996 - val_accuracy: 0.5847 - val_precision: 0.6467 - val_recall: 0.1950
Epoch 36/500
350/350 [==============================] - 2s 5ms/step - loss: 0.6370 - accuracy: 0.6065 - precision: 0.7856 - recall: 0.2104 - val_loss: 0.6807 - val_accuracy: 0.5833 - val_precision: 0.6721 - val_recall: 0.1668
Epoch 37/500
350/350 [==============================] - 1s 3ms/step - loss: 0.6352 - accuracy: 0.6086 - precision: 0.7897 - recall: 0.2146 - val_loss: 0.6862 - val_accuracy: 0.5801 - val_precision: 0.6560 - val_recall: 0.1648
Epoch 38/500
350/350 [==============================] - 1s 4ms/step - loss: 0.6351 - accuracy: 0

350/350 [==============================] - 1s 3ms/step - loss: 0.6227 - accuracy: 0.6224 - precision: 0.7946 - recall: 0.2526 - val_loss: 0.7172 - val_accuracy: 0.5902 - val_precision: 0.6269 - val_recall: 0.2482
Epoch 71/500
350/350 [==============================] - 1s 3ms/step - loss: 0.6233 - accuracy: 0.6189 - precision: 0.7863 - recall: 0.2467 - val_loss: 0.6820 - val_accuracy: 0.5755 - val_precision: 0.6382 - val_recall: 0.1578
Epoch 72/500
350/350 [==============================] - 1s 3ms/step - loss: 0.6212 - accuracy: 0.6201 - precision: 0.7895 - recall: 0.2486 - val_loss: 0.6884 - val_accuracy: 0.5957 - val_precision: 0.6750 - val_recall: 0.2171
Epoch 73/500
350/350 [==============================] - 1s 3ms/step - loss: 0.6226 - accuracy: 0.6227 - precision: 0.7991 - recall: 0.2509 - val_loss: 0.6802 - val_accuracy: 0.5893 - val_precision: 0.7025 - val_recall: 0.1709
Epoch 74/500
350/350 [==============================] - 1s 4ms/step - loss: 0.6213 - accuracy: 0.6246 - preci

350/350 [==============================] - 1s 3ms/step - loss: 0.6148 - accuracy: 0.6390 - precision: 0.8149 - recall: 0.2886 - val_loss: 0.6953 - val_accuracy: 0.5929 - val_precision: 0.6636 - val_recall: 0.2161
Epoch 107/500
350/350 [==============================] - 1s 3ms/step - loss: 0.6114 - accuracy: 0.6325 - precision: 0.8090 - recall: 0.2735 - val_loss: 0.6889 - val_accuracy: 0.6016 - val_precision: 0.6855 - val_recall: 0.2322
Epoch 108/500
350/350 [==============================] - 1s 3ms/step - loss: 0.6111 - accuracy: 0.6363 - precision: 0.8113 - recall: 0.2829 - val_loss: 0.7237 - val_accuracy: 0.5957 - val_precision: 0.6443 - val_recall: 0.2513
Epoch 109/500
350/350 [==============================] - 1s 3ms/step - loss: 0.6118 - accuracy: 0.6359 - precision: 0.8057 - recall: 0.2851 - val_loss: 0.7056 - val_accuracy: 0.6062 - val_precision: 0.6781 - val_recall: 0.2583
Epoch 110/500
350/350 [==============================] - 1s 3ms/step - loss: 0.6136 - accuracy: 0.6346 - p

350/350 [==============================] - 1s 3ms/step - loss: 0.6055 - accuracy: 0.6461 - precision: 0.8225 - recall: 0.3038 - val_loss: 0.6932 - val_accuracy: 0.5952 - val_precision: 0.6657 - val_recall: 0.2241
Epoch 143/500
350/350 [==============================] - 1s 3ms/step - loss: 0.6053 - accuracy: 0.6442 - precision: 0.8184 - recall: 0.3009 - val_loss: 0.7083 - val_accuracy: 0.5829 - val_precision: 0.6875 - val_recall: 0.1548
Epoch 144/500
350/350 [==============================] - 1s 3ms/step - loss: 0.6050 - accuracy: 0.6464 - precision: 0.8289 - recall: 0.3009 - val_loss: 0.7576 - val_accuracy: 0.6044 - val_precision: 0.6267 - val_recall: 0.3256
Epoch 145/500
350/350 [==============================] - 1s 3ms/step - loss: 0.6061 - accuracy: 0.6401 - precision: 0.8105 - recall: 0.2940 - val_loss: 0.7028 - val_accuracy: 0.5984 - val_precision: 0.6715 - val_recall: 0.2322
Epoch 146/500
350/350 [==============================] - 1s 3ms/step - loss: 0.6061 - accuracy: 0.6422 - p

...

350/350 [==============================] - 1s 3ms/step - loss: 0.5876 - accuracy: 0.6643 - precision: 0.8408 - recall: 0.3423 - val_loss: 0.7417 - val_accuracy: 0.6035 - val_precision: 0.6684 - val_recall: 0.2573
Epoch 287/500
350/350 [==============================] - 1s 3ms/step - loss: 0.5890 - accuracy: 0.6664 - precision: 0.8381 - recall: 0.3494 - val_loss: 0.7356 - val_accuracy: 0.5962 - val_precision: 0.6717 - val_recall: 0.2221
Epoch 288/500
350/350 [==============================] - 1s 3ms/step - loss: 0.5872 - accuracy: 0.6620 - precision: 0.8364 - recall: 0.3388 - val_loss: 0.7405 - val_accuracy: 0.5971 - val_precision: 0.6638 - val_recall: 0.2342
Epoch 289/500
350/350 [==============================] - 1s 3ms/step - loss: 0.5884 - accuracy: 0.6635 - precision: 0.8397 - recall: 0.3408 - val_loss: 0.7293 - val_accuracy: 0.5989 - val_precision: 0.6901 - val_recall: 0.2171
Epoch 290/500
350/350 [==============================] - 1s 3ms/step - loss: 0.5875 - accuracy: 0.6630 - p

350/350 [==============================] - 1s 3ms/step - loss: 0.5849 - accuracy: 0.6679 - precision: 0.8433 - recall: 0.3502 - val_loss: 0.7465 - val_accuracy: 0.6016 - val_precision: 0.6640 - val_recall: 0.2543
Epoch 323/500
350/350 [==============================] - 1s 3ms/step - loss: 0.5820 - accuracy: 0.6669 - precision: 0.8445 - recall: 0.3467 - val_loss: 0.7802 - val_accuracy: 0.5994 - val_precision: 0.6471 - val_recall: 0.2653
Epoch 324/500
350/350 [==============================] - 1s 3ms/step - loss: 0.5869 - accuracy: 0.6641 - precision: 0.8306 - recall: 0.3480 - val_loss: 0.7405 - val_accuracy: 0.6049 - val_precision: 0.6941 - val_recall: 0.2372
Epoch 325/500
350/350 [==============================] - 1s 3ms/step - loss: 0.5875 - accuracy: 0.6658 - precision: 0.8340 - recall: 0.3504 - val_loss: 0.7532 - val_accuracy: 0.5897 - val_precision: 0.6634 - val_recall: 0.2020
Epoch 326/500
350/350 [==============================] - 1s 3ms/step - loss: 0.5843 - accuracy: 0.6675 - p

350/350 [==============================] - 1s 3ms/step - loss: 0.5846 - accuracy: 0.6689 - precision: 0.8409 - recall: 0.3544 - val_loss: 0.7763 - val_accuracy: 0.6035 - val_precision: 0.6625 - val_recall: 0.2643
Epoch 359/500
350/350 [==============================] - 1s 3ms/step - loss: 0.5819 - accuracy: 0.6749 - precision: 0.8508 - recall: 0.3640 - val_loss: 0.7586 - val_accuracy: 0.5952 - val_precision: 0.6504 - val_recall: 0.2412
Epoch 360/500
350/350 [==============================] - 1s 3ms/step - loss: 0.5836 - accuracy: 0.6645 - precision: 0.8426 - recall: 0.3415 - val_loss: 0.8222 - val_accuracy: 0.6026 - val_precision: 0.6433 - val_recall: 0.2864
Epoch 361/500
350/350 [==============================] - 1s 3ms/step - loss: 0.5843 - accuracy: 0.6675 - precision: 0.8410 - recall: 0.3507 - val_loss: 0.8010 - val_accuracy: 0.6071 - val_precision: 0.6378 - val_recall: 0.3186
Epoch 362/500
350/350 [==============================] - 1s 3ms/step - loss: 0.5826 - accuracy: 0.6698 - p

350/350 [==============================] - 1s 3ms/step - loss: 0.5804 - accuracy: 0.6724 - precision: 0.8477 - recall: 0.3593 - val_loss: 0.7769 - val_accuracy: 0.6058 - val_precision: 0.6558 - val_recall: 0.2834
Epoch 395/500
350/350 [==============================] - 1s 3ms/step - loss: 0.5799 - accuracy: 0.6694 - precision: 0.8417 - recall: 0.3551 - val_loss: 0.7840 - val_accuracy: 0.6049 - val_precision: 0.6392 - val_recall: 0.3045
Epoch 396/500
350/350 [==============================] - 1s 3ms/step - loss: 0.5817 - accuracy: 0.6703 - precision: 0.8461 - recall: 0.3549 - val_loss: 0.7865 - val_accuracy: 0.6081 - val_precision: 0.6534 - val_recall: 0.2975
Epoch 397/500
350/350 [==============================] - 1s 3ms/step - loss: 0.5801 - accuracy: 0.6745 - precision: 0.8559 - recall: 0.3600 - val_loss: 0.8118 - val_accuracy: 0.6058 - val_precision: 0.6650 - val_recall: 0.2714
Epoch 398/500
350/350 [==============================] - 1s 3ms/step - loss: 0.5783 - accuracy: 0.6728 - p

KeyboardInterrupt: 

## Train 1
Deep Neural Network:
* Input Layer: 431 nodes
* Hidden Layer 1: 431 nodes
* Hidden Layer 2: 40 nodes
* Ouput Layer: 1 node

Hyper-parameters:
* Loss Function: BinaryCrossEntropy
* Activation Function: Relu
* Optimizer Function: RMSprop
* Learning Rate: 0.001 
* Epochs: 500
* Batch_Size: 25
* Classification Threshold: 0.7
* Regularization: L2
* Regularization Lambda: 0.06

In [ ]:
# Define the functions that create and train a model.
def create_model(my_learning_rate, my_metrics):
    """Create and compile a simple classification model."""
    # Discard any pre-existing version of the model.
    model = None

    # Most simple tf.keras models are sequential.
    model = tf.keras.models.Sequential()

    # Add the input layer of 431 nodes
    model.add(tf.keras.layers.Dense(units=431, input_shape=(431,)))
              
    # Implement L2 regularization in the first hidden layer.
    model.add(tf.keras.layers.Dense(units=431, 
                                  activation='relu',
                                  kernel_regularizer=tf.keras.regularizers.l2(0.06),
                                  name='Hidden1'))
    
    # Implement L2 regularization in the second hidden layer.
    model.add(tf.keras.layers.Dense(units=40, 
                                  activation='relu',
                                  kernel_regularizer=tf.keras.regularizers.l2(0.06),
                                  name='Hidden2'))

    # Funnel the regression value through a sigmoid function.
    model.add(tf.keras.layers.Dense(units=1, input_shape=(1,),
                                  activation=tf.sigmoid,
                                  name='Output'))

    # Call the compile method to construct the layers into a model that
    # TensorFlow can execute.  Notice that we're using a different loss
    # function for classification than for regression.    
    model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=my_learning_rate),                                                   
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=my_metrics)

    return model        
              
def train_model(model, features, label, epochs, label_name,
                batch_size=None, my_validation_split=0.1,
                validation_data=None, shuffle=True):
    """Feed a dataset into the model in order to train it."""

    # The x parameter of tf.keras.Model.fit can be a list of arrays.
    history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, shuffle=shuffle, validation_data=validation_data)

    # The list of epochs is stored separately from the rest of history.
    epochs = history.epoch

    # Isolate the classification metric for each epoch.
    hist = pd.DataFrame(history.history)

    return epochs, hist  

print("Defined the create_model and train_model functions.")  

In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.001
epochs = 500
batch_size = 25
classification_threshold = 0.70
label_name = "label"

# Here is the updated definition of METRICS:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(
          name='accuracy', threshold=classification_threshold),
      tf.keras.metrics.Precision(
          thresholds=classification_threshold, name='precision'),
      tf.keras.metrics.Recall(
          thresholds=classification_threshold, name="recall"),
]

# Establish the model's topography.
my_model = create_model(learning_rate, METRICS)

my_model.summary()

# Train the model on the training set.
epochs, hist = train_model(my_model, x_train, y_train, epochs, 
                          label_name, batch_size, validation_data=(x_val, y_val))

# Plot metrics vs. epochs
list_of_metrics_to_plot = ['accuracy', "precision", "recall", 'val_accuracy', 'val_precision', 'val_recall'] 
plot_curve(epochs, hist, list_of_metrics_to_plot)
plot_curve(epochs, hist, ['loss', 'val_loss'])

training_performance =  my_model.evaluate(x_train, y_train, verbose=0)
print('Training Performance')
print('---------------------------------')
print('loss: ', training_performance[0])
print('accuracy: ', training_performance[1])
print('precision: ', training_performance[2])
print('recall: ', training_performance[3])
print()

validation_performance =  my_model.evaluate(x_val, y_val, verbose=0)
print('Validation Performance')
print('---------------------------------')
print('loss: ', validation_performance[0])
print('accuracy: ', validation_performance[1])
print('precision: ', validation_performance[2])
print('recall: ', validation_performance[3])

## Train 2
Deep Neural Network:
* Input Layer: 431 nodes
* Hidden Layer 1: 431 nodes
* Hidden Layer 2: 40 nodes
* Ouput Layer: 1 node

Hyper-parameters:
* Loss Function: BinaryCrossEntropy
* Activation Function: Relu
* Optimizer Function: Adam
* Learning Rate: 0.001 
* Epochs: 500
* Batch_Size: 25
* Classification Threshold: 0.7
* Regularization: L2
* Regularization Lambda: 0.05

In [ ]:
# Define the functions that create and train a model.
def create_model(my_learning_rate, my_metrics):
    """Create and compile a simple classification model."""
    # Discard any pre-existing version of the model.
    model = None

    # Most simple tf.keras models are sequential.
    model = tf.keras.models.Sequential()

    # Add the input layer of 431 nodes
    model.add(tf.keras.layers.Dense(units=431, input_shape=(431,)))
              
    # Implement L2 regularization in the first hidden layer.
    model.add(tf.keras.layers.Dense(units=431, 
                                  activation='relu',
                                  kernel_regularizer=tf.keras.regularizers.l2(0.005),
                                  name='Hidden1'))
    
    # Implement L2 regularization in the second hidden layer.
    model.add(tf.keras.layers.Dense(units=40, 
                                  activation='relu',
                                  kernel_regularizer=tf.keras.regularizers.l2(0.005),
                                  name='Hidden2'))
    
    # Implement L2 regularization in the third hidden layer.
    model.add(tf.keras.layers.Dense(units=20, 
                                  activation='relu',
                                  kernel_regularizer=tf.keras.regularizers.l2(0.005),
                                  name='Hidden3'))

    # Funnel the regression value through a sigmoid function.
    model.add(tf.keras.layers.Dense(units=1, input_shape=(1,),
                                  activation=tf.sigmoid,
                                  name='Output'))

    # Call the compile method to construct the layers into a model that
    # TensorFlow can execute.  Notice that we're using a different loss
    # function for classification than for regression.    
    model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=my_learning_rate),                                                   
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=my_metrics)

    return model        
              
def train_model(model, features, label, epochs, label_name,
                batch_size=None, my_validation_split=0.1,
                validation_data=None, shuffle=True):
    """Feed a dataset into the model in order to train it."""

    # The x parameter of tf.keras.Model.fit can be a list of arrays.
    history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, shuffle=shuffle, validation_data=validation_data)

    # The list of epochs is stored separately from the rest of history.
    epochs = history.epoch

    # Isolate the classification metric for each epoch.
    hist = pd.DataFrame(history.history)

    return epochs, hist  

print("Defined the create_model and train_model functions.")  

In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.002
epochs = 100
batch_size = 50
classification_threshold = 0.70
label_name = "label"

# Here is the updated definition of METRICS:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(
          name='accuracy', threshold=classification_threshold),
      tf.keras.metrics.Precision(
          thresholds=classification_threshold, name='precision'),
      tf.keras.metrics.Recall(
          thresholds=classification_threshold, name="recall"),
]

# Establish the model's topography.
my_model = create_model(learning_rate, METRICS)

my_model.summary()

# Train the model on the training set.
epochs, hist = train_model(my_model, x_train, y_train, epochs, 
                          label_name, batch_size, validation_data=(x_val, y_val))

# Plot metrics vs. epochs
list_of_metrics_to_plot = ['accuracy', "precision", "recall", 'val_accuracy', 'val_precision', 'val_recall'] 
plot_curve(epochs, hist, list_of_metrics_to_plot)
plot_curve(epochs, hist, ['loss', 'val_loss'])

training_performance =  my_model2.evaluate(x_train, y_train, verbose=0)
print('Training Performance')
print('---------------------------------')
print('loss: ', training_performance[0])
print('accuracy: ', training_performance[1])
print('precision: ', training_performance[2])
print('recall: ', training_performance[3])
print()

validation_performance =  my_model2.evaluate(x_val, y_val, verbose=0)
print('Validation Performance')
print('---------------------------------')
print('loss: ', validation_performance[0])
print('accuracy: ', validation_performance[1])
print('precision: ', validation_performance[2])
print('recall: ', validation_performance[3])